In [ ]:
%matplotlib ipympl

from experimental.beacon_sim.experiment_results_pb2 import ExperimentResult
from experimental.beacon_sim.world_map_config_pb2 import WorldMapConfig
from experimental.beacon_sim.mapped_landmarks_pb2 import MappedLandmarks
import planning.probabilistic_road_map_python as prmp
import itertools
import tqdm

import pandas as pd
import pandas.io.formats.style

from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets
import matplotlib as mpl
import re

mpl.style.use('ggplot')

import seaborn as sns

from pathlib import Path

import numpy as np

In [ ]:

logs = {
    'expected_det': {
        'results': Path('/home/erick/scratch/beacon_sim/20240711/reprocessed_results/'),
        'configs': Path('/home/erick/scratch/beacon_sim/20240711/experiment_configs')
    },
    'prob_mass_in_region': {
        'results': Path('/home/erick/scratch/beacon_sim/20240716/reprocessed_results/'),
        'configs': Path('/home/erick/scratch/beacon_sim/20240716/experiment_configs')
    }
}

def protobuf_to_df(pb):
    out = defaultdict(list)
    trial_samples = defaultdict(list)

    for result in pb.results:
        planner_name = pb.planner_names[result.planner_id]
        m = re.match(r'(.*?) BRM\s?(\d*)', planner_name)
        out['elapsed_time'].append(result.elapsed_time.count / 1e9)
        out['trial_id'].append(result.trial_id)
        out['planner_id'].append(result.planner_id)
        out['planner_name'].append(planner_name)
        out['planner_type'].append(m.group(1))
        out['planner_size'].append(m.group(2) if len(m.group(2)) > 0 else '1')
        out['start'].append(np.array(pb.start_goal[result.trial_id].start.data))
        out['goal'].append(np.array(pb.start_goal[result.trial_id].goal.data))
        out['timeout'].append(pb.experiment_config.plan_timeout_s)
        if result.HasField('plan'):
            out['plan'].append(list(result.plan.nodes))
            out['log_prob_mass'].append(result.plan.log_prob_mass)
            out['expected_size'].append(result.plan.expected_size)
            out['expected_determinant'].append(result.plan.average_plan_metrics.expected_determinant)
            out['expected_position_determinant'].append(result.plan.average_plan_metrics.expected_position_determinant)
            out['prob_mass_in_region'].append(1-result.plan.average_plan_metrics.prob_mass_in_region)

            for sample_id, sampled_metrics in enumerate(result.plan.sampled_plan_metrics):
                trial_samples['sample_id'].append(sample_id)
                trial_samples['trial_id'].append(result.trial_id)
                trial_samples['planner_id'].append(result.planner_id)
                trial_samples['expected_determinant'].append(sampled_metrics.expected_determinant)
                trial_samples['expected_position_determinant'].append(sampled_metrics.expected_position_determinant)
                trial_samples['prob_mass_in_region'].append(1-sampled_metrics.prob_mass_in_region)
        else:
            out['plan'].append(None)
            out['log_prob_mass'].append(None)
            out['expected_size'].append(None)
            out['expected_determinant'].append(None)
            out['expected_position_determinant'].append(None)
            out['prob_mass_in_region'].append(None)
    return pd.DataFrame(out), pd.DataFrame(trial_samples)

df = pd.DataFrame()
samples_df_by_exp_name = {}
for objective, paths in logs.items():

    results_dir = paths['results']
    configs_dir = paths['configs']

    results_files = sorted(results_dir.glob("*.pb"))
    
    protos_by_exp = {}
    configs_by_exp = {}
    print('reading from objective', objective)
    for f in tqdm.tqdm(results_files):
        with open(f, 'rb') as file_in:
            data = ExperimentResult()
            data.ParseFromString(file_in.read())
            protos_by_exp[f.stem] = data
    
            map_config_path = configs_dir / f.stem / data.experiment_config.map_config_path
            road_map_path = configs_dir / f.stem / data.experiment_config.road_map_path
            mapped_landmark_path = configs_dir / f.stem / data.experiment_config.ekf_state_path
    
            wmc = WorldMapConfig()
            wmc.ParseFromString(map_config_path.read_bytes())
    
            assert road_map_path.exists()
            map_proto = MappedLandmarks()
            map_proto.ParseFromString(mapped_landmark_path.read_bytes())
            configs_by_exp[f.stem] = {
                'road_map': prmp.RoadMap.from_proto_string(road_map_path.read_bytes()),
                'world_map_config': wmc,
                'mapped_landmarks': map_proto,
            }


    
    for name, proto in tqdm.tqdm(protos_by_exp.items()):
        sub_df, sub_samples_df = protobuf_to_df(proto)
    
        sub_df['exp_name'] = name
        sub_df['exp_type'] = name.split('_')[0]
        sub_df['objective'] = objective
        df = pd.concat([df, sub_df])
        samples_df_by_exp_name[(objective,  name)] = sub_samples_df
        
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
plt.figure(figsize=(12, 24))

ax = plt.subplot(4,2,1)
for i, (exp_type, objective) in enumerate(itertools.product(['indep', 'spatial', 'mutex', 'semantic'], ['expected_det', 'prob_mass_in_region'])):
    plt.subplot(4, 2, i+1)
    mask = np.logical_and(df['objective'] == objective, df['exp_type'] == exp_type)
    plt.hist2d(df[mask]['expected_determinant'], df[mask]['prob_mass_in_region'], bins=[np.logspace(-16, 2), np.linspace(0.0, 1.0)],
            norm='log')
    plt.colorbar()
    plt.xscale('log')
    plt.xlabel('Expected Determinant')
    plt.ylabel('Prob Mass in Region')
    plt.xlim(1e-12, 10)
    plt.title(f"{objective} {exp_type}")
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 24))

ax = plt.subplot(4,2,1)
for i, (exp_type, objective) in enumerate(itertools.product(['indep', 'spatial', 'mutex', 'semantic'], ['expected_det', 'prob_mass_in_region'])):
    plt.subplot(4, 2, i+1)
    mask = np.logical_and(df['objective'] == objective, df['exp_type'] == exp_type)
    sns.scatterplot(data=df[mask], x='expected_determinant', y='expected_position_determinant', hue='exp_type')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(1e-12, 10)
    plt.title(f"{objective} {exp_type}")
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 24))

ax = plt.subplot(4,2,1)
for i, (exp_type, objective) in enumerate(itertools.product(['indep', 'spatial', 'mutex', 'semantic'], ['expected_det', 'prob_mass_in_region'])):
    plt.subplot(4, 2, i+1)
    mask = np.logical_and(df['objective'] == objective, df['exp_type'] == exp_type)
    sns.scatterplot(data=df[mask], x='expected_position_determinant', y='prob_mass_in_region', hue='exp_type')
    plt.xscale('log')
    plt.xlim(1e-12, 1e4)
    plt.title(f"{objective} {exp_type}")
    plt.tight_layout()

In [ ]:
low_prob_mask = df['prob_mass_in_region'] < 0.1
low_prob_df = df[low_prob_mask]
lowest_det_idx = low_prob_df['expected_position_determinant'].argmin()
lowest_det_trial = low_prob_df.iloc[lowest_det_idx]

In [ ]:

plt.figure(figsize=(16, 12))

for i, (key, sub_df) in enumerate(df.groupby(['planner_name'])):
    print(key, len(sub_df))
    prob_mass_mask = sub_df['objective'] == 'prob_mass_in_region'
    expected_det_mask = sub_df['objective'] == 'expected_det'
    prob_mass_df = sub_df[prob_mass_mask]
    expected_det_df = sub_df[expected_det_mask]
    joint = prob_mass_df.merge(expected_det_df, on=['exp_name', 'trial_id'], how='inner', suffixes=['_obj_pm', '_obj_ed'])

    plt.subplot(3, 3, i+1)
    plt.hist2d(joint['prob_mass_in_region_obj_pm'], joint['prob_mass_in_region_obj_ed'], bins=30, norm='log')
    plt.xlabel('P(Goal) Optim: P(Goal)')
    plt.ylabel('P(Goal) Optim: E[$|\Sigma|$]')
    plt.colorbar()
    plt.plot([0, 1], [0,1], linestyle='dashed', color='black')
    plt.title(f"{key}")
    
    
plt.tight_layout()

In [ ]:

prob_mass_mask = df['objective'] == 'prob_mass_in_region'
expected_det_mask = df['objective'] == 'expected_det'
prob_mass_df = df[prob_mass_mask]
expected_det_df = df[expected_det_mask]
joint = prob_mass_df.merge(expected_det_df, on=['exp_name', 'trial_id', 'planner_name', 'index', 'planner_id', 'planner_type', 'planner_size', 'timeout', 'exp_type'], how='inner', suffixes=['_obj_pm', '_obj_ed'])

In [ ]:
len(joint)

In [ ]:
joint

In [ ]:
joint['prob_delta'] = joint['prob_mass_in_region_obj_pm'] - joint['prob_mass_in_region_obj_ed']

In [ ]:
min_idx = joint['prob_delta'].argmin()
joint.loc[min_idx]